<a href="https://colab.research.google.com/github/Kagev/DataScience/blob/main/DS_HW_10_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Завдання**
На цьому тижні ми вивчили як працюють рекомендаційні системи. Пропонуємо вам познайомитись з бібліотекою surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Візьміть датасет movielens і побудуйте модель матричної факторизації. У даній бібліотеці він має назву SVD. Підберіть найкращі параметри за допомогою крос-валідації, також поекспериментуйте з іншими алгоритмами розрахунків (SVD++, NMF) і оберіть той, який буде оптимальним.

Підказки як саме побудувати дану модель ви знайдете в документації до даної бібліотеки.

 **Додаткове завдання з зірочкою**
Для більшого заглиблення в роботу алгоритму, пропонуємо реалізувати алгоритм колабораційної фільтрації з нуля. Для цього ми можемо скористатись нашою домашньою роботою з 3-ого модуля. Якщо ми модифікуємо функцію втрат та розрахунок градієнтів, то зможемо побудувати алгоритм матричної факторизації.

Тут ви можете побачити формули та завантаження датасету. А ось посилання на назви фільмів та на рейтинги.

Вдалої роботи!

In [14]:
!pip install scikit-surprise

In [15]:
import numpy as np
from surprise import Dataset, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV


In [13]:


# Завантажуємо вбудований датасет MovieLens (ml-100k)
data = Dataset.load_builtin('ml-100k')

# Створюємо модель на основі SVD
algo_svd = SVD()

# Використовуємо крос-валідацію для оцінки моделі
print("Крос-валідація для SVD:")
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Створюємо модель на основі SVD++
algo_svdpp = SVDpp()

print("\nКрос-валідація для SVD++:")
cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Створюємо модель на основі NMF
algo_nmf = NMF()

print("\nКрос-валідація для NMF:")
cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Крос-валідація для SVD:
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9455  0.9356  0.9280  0.9371  0.9362  0.9365  0.0056  
MAE (testset)     0.7427  0.7379  0.7321  0.7373  0.7372  0.7374  0.0034  
Fit time          0.78    1.12    0.80    0.76    0.80    0.85    0.14    
Test time         0.08    0.24    0.08    0.15    0.07    0.12    0.07    

Крос-валідація для SVD++:
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9201  0.9200  0.9144  0.9181  0.9304  0.9206  0.0053  
MAE (testset)     0.7224  0.7207  0.7171  0.7214  0.7280  0.7219  0.0035  
Fit time          24.40   23.91   25.82   25.78   24.33   24.85   0.80    
Test time         3.50    4.13    3.58    3.75    3.45    3.68    0.25    

Крос-валідація для NMF:
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

          

{'test_rmse': array([0.96064089, 0.96131195, 0.96590162, 0.96477016, 0.96206602]),
 'test_mae': array([0.75625319, 0.75612647, 0.75949207, 0.75805923, 0.75759337]),
 'fit_time': (1.5674197673797607,
  1.069087266921997,
  1.2971439361572266,
  0.9964685440063477,
  1.2960810661315918),
 'test_time': (0.2428891658782959,
  0.06467008590698242,
  0.15397429466247559,
  0.06362271308898926,
  0.14857745170593262)}

In [16]:

# Підбір параметрів для SVD
param_grid = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005, 0.01], 'reg_all': [0.2, 0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Тренуємо модель
gs.fit(data)

# Виводимо найкращі результати
print("Найкращі параметри для SVD:")
print(gs.best_params['rmse'])

# Те саме можна зробити для інших моделей, таких як SVD++ та NMF


Найкращі параметри для SVD:
{'n_epochs': 20, 'lr_all': 0.01, 'reg_all': 0.2}


Додаткове завдання: Реалізація алгоритму колабораційної фільтрації з нуля

In [17]:
# Ініціалізуємо дані (спрощена версія)
# Припустимо, що у нас є 5 користувачів і 4 фільми
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# Ініціалізуємо кількість факторів
num_users, num_items = R.shape
num_factors = 3

# Ініціалізуємо матриці U (користувачі) і V (фільми)
U = np.random.normal(scale=1./num_factors, size=(num_users, num_factors))
V = np.random.normal(scale=1./num_factors, size=(num_items, num_factors))

# Гіперпараметри
learning_rate = 0.01
regularization = 0.1
num_epochs = 5000

# Функція для обчислення помилки
def compute_loss(R, U, V, regularization):
    loss = 0
    for i in range(num_users):
        for j in range(num_items):
            if R[i, j] > 0:
                loss += (R[i, j] - np.dot(U[i, :], V[j, :])) ** 2
                for k in range(num_factors):
                    loss += (regularization / 2) * (U[i, k] ** 2 + V[j, k] ** 2)
    return loss

# Градієнтний спуск для оновлення матриць U і V
for epoch in range(num_epochs):
    for i in range(num_users):
        for j in range(num_items):
            if R[i, j] > 0:
                # Обчислюємо помилку
                error = R[i, j] - np.dot(U[i, :], V[j, :])
                # Оновлюємо матриці
                U[i, :] += learning_rate * (error * V[j, :] - regularization * U[i, :])
                V[j, :] += learning_rate * (error * U[i, :] - regularization * V[j, :])
    # Виводимо втрату кожні 1000 епох
    if epoch % 1000 == 0:
        print(f"Епоха {epoch}: Втрата = {compute_loss(R, U, V, regularization)}")

# Передбачення
predicted_ratings = np.dot(U, V.T)
print("Передбачені рейтинги:")
print(predicted_ratings)


Епоха 0: Втрата = 135.20819452969795
Епоха 1000: Втрата = 5.528641077943098
Епоха 2000: Втрата = 5.525465232187784
Епоха 3000: Втрата = 5.524250992849268
Епоха 4000: Втрата = 5.523723848517307
Передбачені рейтинги:
[[4.76691827 2.83751951 1.71970169 1.01504162]
 [3.82259287 2.29077319 1.5966771  0.99460474]
 [1.007505   0.98271082 5.79133775 4.71617996]
 [0.99082287 0.89856846 4.73330523 3.84041945]
 [1.13336109 0.98826172 4.85907533 3.92602512]]
